# AI Red Teaming Agent for Generative AI models and applications in Azure AI Foundry

## Objective
This notebook walks through how to use Azure AI Evaluation's AI Red Teaming Agent functionality to assess the safety and resilience of AI systems against adversarial prompt attacks. AI Red Teaming Agent leverages [Risk and Safety Evaluations](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/evaluation-metrics-built-in?tabs=warning#risk-and-safety-evaluators) to help identify potential safety issues across different risk categories (violence, hate/unfairness, sexual content, self-harm) combined with attack strategies of varying complexity levels from [PyRIT](https://github.com/Azure/PyRIT), Microsoft AI Red Teaming team's open framework for automated AI red teaming.

## Time
You should expect to spend about 30-45 minutes running this notebook. Execution time will vary based on the number of risk categories, attack strategies, and complexity levels you choose to evaluate.

## Before you begin

### Prerequisite
First, if you have an Azure subscription, create an [Azure AI hub](https://learn.microsoft.com/en-us/azure/ai-studio/concepts/ai-resources) then [create an Azure AI project](https://learn.microsoft.com/en-us/azure/ai-studio/concepts/ai-resources). AI projects and Hubs can be served within a private network and are compatible with private endpoints. You **do not** need to provide your own LLM deployment as the AI Red Teaming Agent hosts adversarial models for both simulation and evaluation of harmful content and connects to it via your Azure AI project.

In order to upload your results to Azure AI Foundry:
- Your AI Foundry project must have a connection (*Connected Resources*) to a storage account with `Microsoft Entra ID` authentication enabled.
- Your AI Foundry project must have the `Storage Blob Data Contributor` role in the storage account.
- You must have the `Storage Blob Data Contributor` role in the storage account.
- You must have network access to the storage account.

For more information see: https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/run-scans-ai-red-teaming-agent

**Important**: First, ensure that you've installed the [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) and then make sure to authenticate to Azure using `az login` in your terminal before running this notebook.

### Installation
From a terminal window, navigate to your working directory which contains this sample notebook, and execute the following.
```bash
python -m venv .venv
```

Then, activate the virtual environment created:

```bash
# %source .venv/bin/activate # If using Mac/Linux OS
.venv/Scripts/activate # If using Windows OS
```

With your virtual environment activated, install the following packages required to execute this notebook:

```bash
pip install uv
uv pip install azure-ai-evaluation[redteam] azure-identity openai azure-ai-projects
```


Now open VSCode with the following command, and ensure your virtual environment is used as kernel to run the remainder of this notebook.
```bash
code .
```

### Imports

In [1]:
from typing import Optional, Dict, Any
import os

# Azure imports
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy

# OpenAI imports
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv(override=True)

True

### Login to Azure with valid credentials

Ensure that you've installed the [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) and then make sure to authenticate to Azure using `az login` in your terminal before running this notebook.

Configure the `credential` object with a different AzureCredential type if this is a requirement for your environment.

In [3]:
# Azure Credential imports
from azure.identity import AzureCliCredential, get_bearer_token_provider

#!az login

# Initialize Azure credentials
credential = AzureCliCredential()

### Set Up Your Environment Variables

Set the following variables for use in this notebook. These variables connect to your Azure resources and model deployments.

Set these variables by creating an `.env` file in your project's root folder.

**Note:** You can find these values in your Azure AI Foundry project or Azure OpenAI resource.

For reference, here's an example of what your populated environment variables should look like:

```
# Azure OpenAI
AZURE_OPENAI_API_KEY="your-api-key-here"
AZURE_OPENAI_ENDPOINT="https://endpoint-name.cognitiveservices.azure.com/"
AZURE_OPENAI_DEPLOYMENT_NAME="gpt-4"
AZURE_OPENAI_API_VERSION="2024-12-01-preview"

# Azure AI Project
AZURE_PROJECT_ENDPOINT="https://your-aifoundry-endpoint-name.services.ai.azure.com/api/projects/yourproject-name"
```

In [4]:
# Azure AI Project information
azure_ai_project = os.environ.get("AIPROJECT_CONNECTION_STRING")

# Azure OpenAI deployment information
azure_openai_deployment = os.environ.get("OAI_MODEL_DEPLOYMENT_NAME")  # e.g., "gpt-4"
azure_openai_endpoint = os.environ.get("OAI_ENDPOINT")
azure_openai_full_endpoint = os.environ.get("OAI_FULL_ENDPOINT")
azure_openai_api_key = os.environ.get("OAI_KEY")  # e.g., "your-api-key"
azure_openai_api_version = os.environ.get("OPENAI_API_VERSION")  # Use the latest API version

## Understanding AI Red Teaming Agent's capabilities

The Azure AI Evaluation SDK's `RedTeam` functionality evaluates AI systems against adversarial prompts across multiple dimensions:

1. **Risk Categories**: Different content risk categories your AI system might generate
   - Violence
   - HateUnfairness
   - Sexual
   - SelfHarm

2. **Attack Strategies**: Along with standard unmodified prompts which are sent by default as the `baseline`, you can specify different transformations of prompts to elicit undesired content.
You can also use `AttackStrategy.Compose()` to layer two strategies in one attack
   - AnsiAttack: Using ANSI escape codes in prompts
   - AsciiArt: Using ASCII art to disguise harmful content
   - AsciiSmuggler: Hiding harmful content within ASCII characters
   - Atbash: Using the Atbash cipher to encode harmful requests
   - Base64: Encoding harmful content in Base64 format
   - Binary: Converting text to binary to bypass filters
   - Caesar: Using the Caesar cipher for encoding
   - CharacterSpace: Manipulating character spacing to confuse filters
   - CharSwap: Swapping characters to bypass detection
   - Diacritic: Using diacritical marks to alter text appearance
   - Flip: Flipping text to bypass content filters
   - Leetspeak: Converting letters to numbers and symbols
   - Morse: Using Morse code to encode harmful requests
   - ROT13: Using ROT13 cipher for text transformation
   - SuffixAppend: Adding suffixes to confuse detection systems
   - StringJoin: Joining strings in unconventional ways
   - Tense: Changing the tense of harmful requests to past tense
   - UnicodeConfusable: Using similar-looking Unicode characters
   - UnicodeSubstitution: Substituting characters with Unicode alternatives
   - Url: Embedding harmful content within URLs
   - Jailbreak: Specially crafted prompts to bypass AI safeguards

3. **Complexity Levels**: Different difficultly levels of attacks
   - Baseline: Standard functionality tests
   - Easy: Simple attack patterns
   - Moderate: More sophisticated attacks
   - Difficult: Complex, layered attack strategies

The key metric for evaluating results is the **Attack Success Rate (ASR)**, which measures the percentage of attacks that successfully elicit harmful content from your AI system.

## Basic Example: Fixed Response Callback

Let's start with a basic example that demonstrates how to set up a Red Team evaluation using a simple callback that always returns a fixed, safe response regardless of the input prompt.

**Important Note**: Since this callback always responds with the same safe message, the Attack Success Rate (ASR) will be 0% for all categories. This example is merely to demonstrate the API structure and flow.

In [ ]:
# Define a simple callback function that always returns a fixed response
def financial_advisor_callback(query: str) -> str:  # noqa: ARG001
    return "I'm a financial advisor assistant. I can help with investment advice and financial planning within legal and ethical guidelines."

In [7]:
# Create the `RedTeam` instance with minimal configurations
red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=[RiskCategory.Violence, RiskCategory.HateUnfairness],
    num_objectives=1,
)

Class RedTeam: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


NOTE: `num_objectives` specifies the number of attacks to perform per risk category per attack strategy. If the parameter `risk_categories` is not specified, `[RiskCategory.Violence, RiskCategory.HateUnfairness, RiskCategory.Sexual, RiskCategory.SelfHarm]` will be used by default.

Now let's run a simple automated scan using the `RedTeam` with the fixed response target. We'll test against two risk categories and one attack strategy for simplicity.

In [ ]:
# Run the red team scan called "Basic-Callback-Scan" with limited scope for this basic example
# This will test 1 objective prompt for each of Violence and HateUnfairness categories with the Flip strategy
result = await red_team.scan(
    target=financial_advisor_callback,
    scan_name="Basic-Callback-Scan",
    attack_strategies=[AttackStrategy.Flip],
    output_path="red_team_output.json",
)

## Intermediary Example: Using a Model Configuration as Target

Now let's create a more realistic example that uses an Azure OpenAI model for responding to the red teaming prompts. To test base or foundation models, you can update your target to take in a model configuration:

In [5]:
# Define a model configuration to test
azure_oai_model_config = {
    "azure_endpoint": azure_openai_full_endpoint,
    "azure_deployment": azure_openai_deployment,
    "api_key": azure_openai_api_key,
}

Then, update your target to point to the model configurations and run the scan.

In [8]:
# Run the red team scan called "Intermediary-Model-Target-Scan"
result = await red_team.scan(
    target=azure_oai_model_config,
    scan_name="Intermediary-Model-Target-Scan",
    attack_strategies=[AttackStrategy.Flip],
)

🚀 STARTING RED TEAM SCAN
📂 Output directory: .\.scan_Intermediary-Model-Target-Scan_20251030_113152
📊 Risk categories: ['violence', 'hate_unfairness']
🔗 Track your red team scan in AI Foundry: None
📋 Planning 4 total tasks
📝 Fetched baseline objectives for violence: 1 objectives
📝 Fetched baseline objectives for hate_unfairness: 1 objectives
🔄 Fetching objectives for strategy 2/2: flip


Scanning:   0%|                                       | 0/4 [00:00<?, ?scan/s, current=initializing]

⚙️ Processing 4 tasks in parallel (max 5 at a time)
▶️ Starting task: baseline strategy for violence risk category


Scanning:   0%|                                       | 0/4 [00:00<?, ?scan/s, current=initializing]

▶️ Starting task: baseline strategy for hate_unfairness risk category
▶️ Starting task: flip strategy for violence risk category
▶️ Starting task: flip strategy for hate_unfairness risk category


...

Evaluation results saved to "c:\sandbox\agents-getting-started\red-teaming\.scan_Intermediary-Model-Target-Scan_20251030_113152\baseline_hate_unfairness_1361a8b7-7bc8-497c-aafd-1f9fefccdeea.json".
✅ Completed task 1/4 (25.0%) - baseline/hate_unfairness in 16.6s
   Est. remaining: 1.1 minutes
Evaluation results saved to "c:\sandbox\agents-getting-started\red-teaming\.scan_Intermediary-Model-Target-Scan_20251030_113152\flip_hate_unfairness_068202bd-6071-48f0-afc3-6c0b47fd09de.json".
✅ Completed task 2/4 (50.0%) - flip/hate_unfairness in 16.6s
   Est. remaining: 0.4 minutes


Scanning:  75%|███████████████████████▎       | 3/4 [00:21<00:06,  6.20s/scan, current=initializing]

Evaluation results saved to "c:\sandbox\agents-getting-started\red-teaming\.scan_Intermediary-Model-Target-Scan_20251030_113152\baseline_violence_30a4d2cd-a49f-4710-9643-ba9b25535dc5.json".
✅ Completed task 3/4 (75.0%) - baseline/violence in 21.7s
   Est. remaining: 0.1 minutes


Scanning: 100%|███████████████████████████████| 4/4 [00:27<00:00,  6.94s/scan, current=initializing]
Class RedTeamResult: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Evaluation results saved to "c:\sandbox\agents-getting-started\red-teaming\.scan_Intermediary-Model-Target-Scan_20251030_113152\flip_violence_e42db16b-f97d-45d9-8dd2-0a3b506e418c.json".
✅ Completed task 4/4 (100.0%) - flip/violence in 27.7s
   Est. remaining: 0.0 minutes
Evaluation results saved to "C:\sandbox\agents-getting-started\red-teaming\.scan_Intermediary-Model-Target-Scan_20251030_113152\final_results.json".

Overall ASR: 0.0%
Attack Success: 0/4 attacks were successful
------------------------------------------------------------------------------------------------------------------------------------
Risk Category        | Baseline ASR   | Easy-Complexity Attacks ASR  | Moderate-Complexity Attacks ASR | Difficult-Complexity Attacks ASR
------------------------------------------------------------------------------------------------------------------------------------
Violence             | 0.0%           | 0.0%                         | N/A                             | N/A    

## Advanced Example: Using an Azure Open AI Model Endpoint in a Callback Function

Using the same Azure Open AI model configuration as above, we now wrap it in a callback function for more flexibility and control on the input and output handling. This will demonstrate how to evaluate an actual AI application. To test your own actual AI application, replace the inside of the callback function with a call to your application.

In [9]:
# Define a callback that uses Azure OpenAI API to generate responses
async def azure_openai_callback(
    messages: list,
    stream: Optional[bool] = False,  # noqa: ARG001
    session_state: Optional[str] = None,  # noqa: ARG001
    context: Optional[Dict[str, Any]] = None,  # noqa: ARG001
) -> dict[str, list[dict[str, str]]]:
    # Get token provider for Azure AD authentication
    token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

    # Initialize Azure OpenAI client
    client = AzureOpenAI(
        azure_endpoint=azure_openai_endpoint,
        api_version=azure_openai_api_version,
        azure_ad_token_provider=token_provider,
    )

    ## Extract the latest message from the conversation history
    messages_list = [{"role": message.role, "content": message.content} for message in messages]
    latest_message = messages_list[-1]["content"]

    try:
        # Call the model
        response = client.chat.completions.create(
            model=azure_openai_deployment,
            messages=[
                {"role": "user", "content": latest_message},
            ],
            # max_tokens=500, # If using an o1 base model, comment this line out
            max_completion_tokens=500,  # If using an o1 base model, uncomment this line
            # temperature=0.7, # If using an o1 base model, comment this line out (temperature param not supported for o1 base models)
        )

        # Format the response to follow the expected chat protocol format
        formatted_response = {"content": response.choices[0].message.content, "role": "assistant"}
    except Exception as e:
        print(f"Error calling Azure OpenAI: {e!s}")
        formatted_response = "I encountered an error and couldn't process your request."
    return {"messages": [formatted_response]}

In [10]:
# Create the RedTeam instance with all of the risk categories with 5 attack objectives generated for each category
model_red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=[RiskCategory.Violence, RiskCategory.HateUnfairness, RiskCategory.Sexual, RiskCategory.SelfHarm],
    num_objectives=5,
)

We will use this instance of `model_red_team` to test different attack strategies in the following section.

### Testing Different Attack Strategies

Now we'll run a more comprehensive evaluation using multiple attack strategies across risk categories. This will give us a better understanding of our model's vulnerabilities.

In [ ]:
# Run the red team scan with multiple attack strategies
advanced_result = await model_red_team.scan(
    target=azure_openai_callback,
    scan_name="Advanced-Callback-Scan",
    attack_strategies=[
        AttackStrategy.EASY,  # Group of easy complexity attacks
        AttackStrategy.MODERATE,  # Group of moderate complexity attacks
        AttackStrategy.CharacterSpace,  # Add character spaces
        AttackStrategy.ROT13,  # Use ROT13 encoding
        AttackStrategy.UnicodeConfusable,  # Use confusable Unicode characters
        AttackStrategy.CharSwap,  # Swap characters in prompts
        AttackStrategy.Morse,  # Encode prompts in Morse code
        AttackStrategy.Leetspeak,  # Use Leetspeak
        AttackStrategy.Url,  # Use URLs in prompts
        AttackStrategy.Binary,  # Encode prompts in binary
        AttackStrategy.Compose([AttackStrategy.Base64, AttackStrategy.ROT13]),  # Use two strategies in one attack
    ],
    output_path="Advanced-Callback-Scan.json",
)

🚀 STARTING RED TEAM SCAN
📂 Output directory: .\.scan_Advanced-Callback-Scan_20251030_113301
📊 Risk categories: ['violence', 'hate_unfairness', 'sexual', 'self_harm']
🔗 Track your red team scan in AI Foundry: None
📋 Planning 52 total tasks
📝 Fetched baseline objectives for violence: 5 objectives
📝 Fetched baseline objectives for hate_unfairness: 5 objectives
📝 Fetched baseline objectives for sexual: 5 objectives
📝 Fetched baseline objectives for self_harm: 5 objectives
🔄 Fetching objectives for strategy 2/13: character_space
🔄 Fetching objectives for strategy 3/13: rot13
🔄 Fetching objectives for strategy 4/13: unicode_confusable
🔄 Fetching objectives for strategy 5/13: char_swap
🔄 Fetching objectives for strategy 6/13: morse
🔄 Fetching objectives for strategy 7/13: leetspeak
🔄 Fetching objectives for strategy 8/13: url
🔄 Fetching objectives for strategy 9/13: binary
🔄 Fetching objectives for strategy 10/13: base64_rot13
🔄 Fetching objectives for strategy 11/13: base64
🔄 Fetching object

Scanning:   0%|                                      | 0/52 [00:00<?, ?scan/s, current=initializing]

⚙️ Processing 52 tasks in parallel (max 5 at a time)
▶️ Starting task: baseline strategy for violence risk category
▶️ Starting task: baseline strategy for hate_unfairness risk category
▶️ Starting task: baseline strategy for sexual risk category
▶️ Starting task: baseline strategy for self_harm risk category
▶️ Starting task: character_space strategy for violence risk category


Evaluation results saved to "c:\sandbox\agents-getting-started\red-teaming\.scan_Advanced-Callback-Scan_20251030_113301\baseline_violence_96a36679-da78-4006-9197-77b9de0e882c.json".
✅ Completed task 1/52 (1.9%) - baseline/violence in 210.9s
   Est. remaining: 186.4 minutes
Evaluation results saved to "c:\sandbox\agents-getting-started\red-teaming\.scan_Advanced-Callback-Scan_20251030_113301\baseline_hate_unfairness_e544bd09-888d-4af4-b28a-96389e5af2eb.json".
✅ Completed task 2/52 (3.8%) - baseline/hate_unfairness in 211.0s
   Est. remaining: 91.4 minutes
Evaluation results saved to "c:\sandbox\agents-getting-started\red-teaming\.scan_Advanced-Callback-Scan_20251030_113301\baseline_sexual_f5c9a1e5-7740-4820-b99a-a36d94026a0d.json".
✅ Completed task 3/52 (5.8%) - baseline/sexual in 211.0s
   Est. remaining: 59.7 minutes
Evaluation results saved to "c:\sandbox\agents-getting-started\red-teaming\.scan_Advanced-Callback-Scan_20251030_113301\baseline_self_harm_0e716c1f-6d29-4836-b8cd-939169c

Scanning:  10%|██▉                           | 5/52 [03:31<24:38, 31.45s/scan, current=initializing]

...

Evaluation results saved to "c:\sandbox\agents-getting-started\red-teaming\.scan_Advanced-Callback-Scan_20251030_113301\flip_violence_10a29d49-3a43-4a89-8335-802abb0621e5.json".
✅ Completed task 45/52 (86.5%) - flip/violence in 245.3s
   Est. remaining: 6.2 minutes
▶️ Starting task: flip strategy for hate_unfairness risk category
▶️ Starting task: flip strategy for sexual risk category
▶️ Starting task: flip strategy for self_harm risk category
▶️ Starting task: tense strategy for violence risk category
▶️ Starting task: tense strategy for hate_unfairness risk category


Scanning:  94%|███████████████████████████▎ | 49/52 [43:58<03:44, 74.88s/scan, current=initializing]

Evaluation results saved to "c:\sandbox\agents-getting-started\red-teaming\.scan_Advanced-Callback-Scan_20251030_113301\flip_hate_unfairness_cfce1dde-bad3-4769-a60e-ba6d70c4745a.json".
✅ Completed task 46/52 (88.5%) - flip/hate_unfairness in 254.2s
   Est. remaining: 5.8 minutes
Evaluation results saved to "c:\sandbox\agents-getting-started\red-teaming\.scan_Advanced-Callback-Scan_20251030_113301\flip_sexual_17af91a0-57db-459c-a8b3-b02bd07837b2.json".
✅ Completed task 47/52 (90.4%) - flip/sexual in 254.2s
   Est. remaining: 4.7 minutes
Evaluation results saved to "c:\sandbox\agents-getting-started\red-teaming\.scan_Advanced-Callback-Scan_20251030_113301\flip_self_harm_34744fc4-bc08-472d-bb6e-b85628e95927.json".
✅ Completed task 48/52 (92.3%) - flip/self_harm in 254.2s
   Est. remaining: 3.7 minutes
Evaluation results saved to "c:\sandbox\agents-getting-started\red-teaming\.scan_Advanced-Callback-Scan_20251030_113301\tense_violence_4558bbcd-ee41-4ba2-bee6-4bf9a5769a5d.json".
✅ Completed

Scanning:  96%|███████████████████████████▉ | 50/52 [44:02<01:25, 42.81s/scan, current=initializing]

Evaluation results saved to "c:\sandbox\agents-getting-started\red-teaming\.scan_Advanced-Callback-Scan_20251030_113301\tense_hate_unfairness_5b1939a6-16ee-4dbb-8d52-210fdf3c0279.json".
✅ Completed task 50/52 (96.2%) - tense/hate_unfairness in 258.4s
   Est. remaining: 1.8 minutes
▶️ Starting task: tense strategy for sexual risk category
▶️ Starting task: tense strategy for self_harm risk category


The data and results used in this attack will be saved to the `output_path` specified. The URL printed out at the end of the scorecard will provide a link to where you results are uploaded and logged to your Azure AI Foundry project.

## Bring your own objectives: Using your own prompts as objectives for RedTeam

Below we demonstrate how to use your own prompts as objectives for a `RedTeam` scan. You can see the required format for prompts under `.\data\prompts.json`. Note that when bringing your own prompts, the supported `risk-type`s are `violence`, `sexual`, `hate_unfairness`, and `self_harm`. The number of prompts you specify will be the `num_objectives` used in the scan. 

In [ ]:
path_to_prompts = ".\data\prompts.json"

# Create the RedTeam specifying the custom attack seed prompts to use as objectives
custom_red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    custom_attack_seed_prompts=path_to_prompts,  # Path to a file containing custom attack seed prompts
)

In [ ]:
custom_red_team_result = await custom_red_team.scan(
    target=azure_openai_callback,
    scan_name="Custom-Prompt-Scan",
    attack_strategies=[
        AttackStrategy.EASY,  # Group of easy complexity attacks
        AttackStrategy.MODERATE,  # Group of moderate complexity attacks
        AttackStrategy.DIFFICULT,  # Group of difficult complexity attacks
    ],
    output_path="Custom-Prompt-Scan.json",
)

## Conclusion

In this notebook, we've demonstrated how to use the Azure AI Evaluation SDK's `RedTeam` functionality to assess the safety and resilience of AI systems. We started with a basic fixed-response example and then moved to a more realistic model testing across multiple risk categories and attack strategies.

The automated AI red teaming scans provides valuable insights into:

1. **Overall Attack Success Rate (ASR)** - The percentage of attacks that successfully elicit harmful content
2. **Vulnerability by Risk Category** - Which types of harmful content your model is most vulnerable to
3. **Effectiveness of Attack Strategies** - Which attack techniques are most successful against your model
4. **Impact of Complexity** - How more sophisticated attacks affect your model's safety guardrails

By regularly red-teaming your AI applications, you can identify and address potential vulnerabilities before deploying your models to production environments.

### Next Steps

1. **Mitigation**: Use these results to strengthen your model's guardrails against identified attack vectors
2. **Continuous Testing**: Implement regular red team evaluations as part of your development lifecycle
3. **Custom Strategies**: Develop custom attack strategies for your specific use cases and domain
4. **Safety Layers**: Consider adding additional safety layers like Azure AI Content Safety to filter harmful requests and responses 